In [ ]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
import os
import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [ ]:
%pip install gpflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
paper_name = "FairML"

In [ ]:
import os, sys
import errno

# make a directory if it does not exist
def make_dir_if_not_exist(used_path):
    if not os.path.isdir(used_path):
        try:
            os.mkdir(used_path)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise exc
            else:
                raise ValueError(f'{used_path} directoy cannot be created because its parent directory does not exist.')

# make directories if they do not exist

make_dir_if_not_exist("/content/drive/MyDrive/data_papers/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/FairBoosting/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/FairBoosting/SVM")

In [ ]:
def dataset_reader(dataset_name):
    
    if dataset_name == 'loan_defaults':
        # https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

        df = pd.read_excel('/content/drive/MyDrive/Datasets/Credit/credit_data.xls', skiprows=[0])
        df = df.drop(columns=['ID'])

    elif dataset_name in ['adult', 'adult_multi']:
        # https://archive.ics.uci.edu/ml/datasets/adult

        df = pd.read_csv('/content/drive/MyDrive/Datasets/Adult/adult.csv')

    elif dataset_name == 'german_credit':
        # https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)

        german_columns_csv = ['Account Balance', 'Duration of Credit (month)',
              'Payment Status of Previous Credit', 'Purpose', 'Credit Amount',
              'Value Savings/Stocks', 'Length of current employment',
              'Instalment per cent', 'Sex & Marital Status', 'Guarantors',
              'Duration in Current address', 'Most valuable available asset',
              'Age (years)', 'Concurrent Credits', 'Type of apartment',
              'No of Credits at this Bank', 'Occupation', 'No of dependents',
              'Telephone', 'Foreign Worker', 'Creditability']
        df = pd.read_csv('/content/drive/MyDrive/Datasets/German/german.data', delim_whitespace=True, header=None)
        df.columns = german_columns_csv
        # extra mapping
        sex_and_marital_map = {'A91':'Male', 'A93':'Male', 'A94':'Male', 'A92':'Female', 'A95':'Female'}
        df['Sex & Marital Status'] = df['Sex & Marital Status'].map(sex_and_marital_map)
        df.rename({'Sex & Marital Status': 'Sex'}, axis=1, inplace=True)

    elif dataset_name == 'german_credit_multi':
        # https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)

        german_columns_csv = ['Account Balance', 'Duration of Credit (month)',
              'Payment Status of Previous Credit', 'Purpose', 'Credit Amount',
              'Value Savings/Stocks', 'Length of current employment',
              'Instalment per cent', 'Sex & Marital Status', 'Guarantors',
              'Duration in Current address', 'Most valuable available asset',
              'Age (years)', 'Concurrent Credits', 'Type of apartment',
              'No of Credits at this Bank', 'Occupation', 'No of dependents',
              'Telephone', 'Foreign Worker', 'Creditability']
        df = pd.read_csv('/content/drive/MyDrive/Datasets/German/german.data', delim_whitespace=True, header=None)
        df.columns = german_columns_csv

        #df = pd.read_csv('/content/drive/MyDrive/Datasets/German/german_credit.csv')

    elif dataset_name == 'communities_and_crime':
        # http://archive.ics.uci.edu/ml/datasets/communities+and+crime

        with open('/content/drive/MyDrive/Datasets/Communities_and_Crime/communities.names') as f:
            colnames = []
            for line in f.readlines():
                if line[:10] == '@attribute':
                  colnames.append(line.split(' ')[1])
        
        df = pd.read_csv('/content/drive/MyDrive/Datasets/Communities_and_Crime/communities.data', header=None)
        df.columns = colnames
        df.drop(['state', 'county', 'community', 'communityname', 'fold'], axis=1, inplace=True)
        
    elif dataset_name == 'portuguese_marketing':
        # https://archive.ics.uci.edu/ml/datasets/bank+marketing
        df = pd.read_csv('/content/drive/MyDrive/Datasets/PortugueseBankMarketing/bank/bank-full.csv', sep=';')

    elif dataset_name == 'student_performance':
        # https://archive.ics.uci.edu/ml/datasets/student+performance
        df = pd.read_csv('/content/drive/MyDrive/Datasets/Student_Performance/student-por.csv', sep=';')
    
    elif dataset_name == 'compas':
        # https://github.com/propublica/compas-analysis
        df = pd.read_csv('/content/drive/MyDrive/Datasets/COMPAS/compas-scores-two-years.csv')
    
    elif dataset_name == 'framingham':
        raise NotImplementedError('Need to find data')

    elif dataset_name == 'meps':
        # https://github.com/Trusted-AI/AIF360/tree/master/aif360/data/raw/meps
        target_path = '/usr/local/lib/python3.7/dist-packages/aif360/data/raw/meps'
        source_path = '/content/drive/MyDrive/Datasets/MEPS'
        from shutil import copyfile
        if not os.path.exists(os.path.join(target_path, 'h181.csv')):
            copyfile(os.path.join(source_path,'h181.csv'), os.path.join(target_path, 'h181.csv'))
        
        if not os.path.exists(os.path.join(target_path, 'h192.csv')):
            copyfile(os.path.join(source_path,'h192.csv'), os.path.join(target_path, 'h192.csv'))

        # Datasets
        from aif360.datasets import MEPSDataset19
        df = MEPSDataset19().convert_to_dataframe()[0]

    else:
       raise ValueError('Does not yet support this dataset {}'.format(dataset_name))
    
    df = df.reset_index(drop=True)
    return df


def protected_and_target_features(dataset_name):
    
    if dataset_name == 'adult':
        protected_features = ['gender']
        target_feature = 'income'

    elif dataset_name == 'adult_multi':
        protected_features = ['race']
        target_feature = 'income'

    elif dataset_name == 'loan_defaults':
        protected_features = ['SEX']
        target_feature = 'default payment next month'

    elif dataset_name == 'german_credit':
        protected_features = ['Sex']
        target_feature = 'Creditability'

    elif dataset_name == 'german_credit_multi':
        protected_features = ['Sex & Marital Status']
        target_feature = 'Creditability'
    
    elif dataset_name == 'communities_and_crime':
        protected_features = ['racepctblack']
        target_feature = 'ViolentCrimesPerPop'

    elif dataset_name == 'student_performance':
        protected_features = ['sex']
        target_feature = 'G3'

    elif dataset_name == 'portuguese_marketing':
        protected_features = ['marital']
        target_feature = 'y'
    
    elif dataset_name == 'compas':
        raise ValueError("needs to be implemented")
    
    elif dataset_name == 'meps':
        raise ValueError("needs to be implemented")
    else:
        raise ValueError("No such data set")

    return protected_features, target_feature


def preprocess_dataset(dataset_name, drop=False):

    data = dataset_reader(dataset_name)
    cols_dict = {k : v for k,v in data.dtypes.to_dict().items() if v == np.dtype('O')}

    object_cols = sorted([x for x in cols_dict.keys()])
    value_cols = sorted([x for x in data.columns if x not in object_cols])

    #object_cols = sorted([x for x in cols_dict.keys() if x not in protected_features])
    #value_cols = sorted([x for x in data.columns if (x not in object_cols) and (x not in protected_features)])
    #object_cols = [x for x in object_cols if x!=target_feature]
    #value_cols = [x for x in value_cols if x!=target_feature]

    data, drop_list = treat_missing_values(data, drop)
    if len(object_cols) > 0:
        ohe_data = customOneHotEncoding(data, object_cols, drop_list)
        comb_data = pd.concat([data[value_cols], ohe_data], axis=1)
    else:
        comb_data = data[value_cols]

    data = data.reset_index(drop=True)
    comb_data = comb_data.reset_index(drop=True)

    return data, comb_data

def treat_missing_values(data, drop=False):

    special_symbols = ['?']
    drop_list = []
    for col in data.columns:
        for sym in special_symbols:
            if (data[col] == sym).any():
                data[col] = data[col].map(lambda x: x.replace(sym, 'missing'))
                drop_list.append(col)

    if drop:
        data = data.dropna()
    else:
        data = data.fillna('missing')
    
    return data, drop_list


def customOneHotEncoding(data, object_cols, drop_cols):
    
    res = []
    for col in object_cols:
        if col in drop_cols:
            ohe = OneHotEncoder(sparse=False, drop=['missing'])
        else:
            ohe = OneHotEncoder(sparse=False, drop='first')

        temp = ohe.fit_transform(data[col].values.reshape(-1, 1))
        temp = pd.DataFrame(data=temp, columns=ohe.get_feature_names_out([col])) 
        res.append(temp)

    return pd.concat(res, axis=1)

In [ ]:
dataset_name = 'loan_defaults'

In [ ]:
protected_features, target = protected_and_target_features(dataset_name)
protected_feature = protected_features[0]
_, processed_data = preprocess_dataset(dataset_name)

protect_col = [x for x in processed_data.columns if protected_feature in x][0]
target_col = [x for x in processed_data.columns if target in x][0]
y = processed_data[target_col]
gr = processed_data[protect_col]
processed_data.drop(columns=[protect_col, target_col], inplace=True)

sc = StandardScaler()
processed_data = sc.fit_transform(processed_data)

In [ ]:
if dataset_name == 'german_credit':
    y.loc[y==2] = -1
elif dataset_name in ['adult', 'loan_defaults']:
    y.loc[y==0] = -1

In [ ]:
class MyDataset(Dataset):
    """
    MovieLens 20M Dataset
    Data preparation
        treat samples with a rating less than 3 as negative samples
    :param dataset_path: MovieLens dataset path
    Reference:
        https://grouplens.org/datasets/movielens
    """

    def __init__(self, X, y, gr):

        self.data = X
        self.targets = y
        self.groups = gr

    def __len__(self):
        return self.targets.shape[0]

    def __getitem__(self, index):
        return self.data[index], self.targets[index], self.groups[index]

In [ ]:
dataset = MyDataset(processed_data, y.values, gr.values)

In [ ]:
train_length = int(len(dataset) * 0.8)
test_length = len(dataset) - train_length
train_dataset, test_dataset = torch.utils.data.random_split(dataset, (train_length, test_length), generator=torch.Generator().manual_seed(42))

In [ ]:
batch_size = 100
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
ALPHA_GRID = [1e-5, 1e-3, 1e-2, 1e-1, 5e-1, 1, 1e1, 3e1, 1e2, 1e3, 3e3, 1e4]
FILEPATH = f'/content/drive/MyDrive/data_papers/{paper_name}/FairBoosting/SVM'

In [ ]:
import gpflow
from scipy.optimize import minimize, LinearConstraint
from functools import partial

In [ ]:
type(w.data)

In [ ]:
type(y)

In [ ]:
def func_to_min(bws,w1,w2,n,y,mask,target,alpha,C):
  b = torch.from_numpy(np.array(bws[-1]))
  w = torch.from_numpy(bws[:-1])
  n1 = int(w1*n)
  loss1 = torch.mean(torch.maximum(torch.zeros(n1), 1 - target[mask]*y[mask]))
  loss2 = torch.mean(torch.maximum(torch.zeros(n-n1), 1 - target[~mask]*y[~mask]))
  max_loss = torch.maximum(loss1, loss2)
  loss = max_loss + 1/alpha * torch.log(w1*torch.exp(alpha*(loss1-max_loss)) + w2*torch.exp(alpha*(loss2-max_loss))) + C/2*torch.dot(w, w) 
  return(loss)
n1

In [ ]:
svc = LinearSVC(max_iter=5000, C=1)

In [ ]:
svc.fit(train_dataset.dataset.data[train_dataset.indices], train_dataset.dataset.targets[train_dataset.indices])

In [ ]:
svc.coef_

In [ ]:
C = 30
lr = 1e-2
L = 0.1
num_epochs = 1000

for alpha in ALPHA_GRID:

    w = torch.autograd.Variable(torch.Tensor(svc.coef_.flatten()), requires_grad=True)
    b = torch.autograd.Variable(torch.Tensor(svc.intercept_), requires_grad=True)
    optimizer = torch.optim.SGD([w, b], lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
    curr_loss = np.inf

    device = 'cpu'
    for epoch in tqdm(range(num_epochs)):
        for i, (fields, target, gr) in enumerate(train_data_loader):
            fields, target, gr = fields.to(device), target.to(device), gr.to(device)

            y = torch.matmul(fields.float(), w) + b
            mask = (gr==1)

            n1 = y[mask].size()[0]
            n = y.size()[0]
            w1 = n1 / n
            w2 = 1 - w1

            loss1 = torch.mean(torch.maximum(torch.zeros(n1), 1 - target[mask]*y[mask]))
            loss2 = torch.mean(torch.maximum(torch.zeros(n-n1), 1 - target[~mask]*y[~mask]))
            #loss1 = torch.mean(torch.log(1 + torch.exp(- target[mask]*y[mask])))
            #loss2 = torch.mean(torch.log(1 + torch.exp(- target[~mask]*y[~mask])))
            
            max_loss = torch.maximum(loss1, loss2)
            total_loss = max_loss + 1/alpha * torch.log(w1*torch.exp(alpha*(loss1-max_loss))+ w2*torch.exp(alpha*(loss2-max_loss)))

            loss = C * total_loss + 0.001 * torch.mean(torch.dot(w, w)) + L * torch.mean(torch.abs(w))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #w.data -= step_size * w.grad.data # step
            #b.data -= step_size * b.grad.data # step

            #w.grad.data.zero_()
            #b.grad.data.zero_()
        
        if curr_loss > total_loss.detach().item():
            curr_loss = total_loss.detach().item()
            w_best = w.detach().numpy().copy()
            b_best = b.detach().numpy().copy()
        else:
            scheduler.step()
    print(alpha, "--->", np.corrcoef(w.detach().numpy(), svc.coef_.flatten())[0][1], "--->", np.corrcoef(w_best, svc.coef_.flatten())[0][1])

    #w = torch.autograd.Variable(torch.Tensor(w_best), requires_grad=True)
    #b = torch.autograd.Variable(torch.Tensor(b_best), requires_grad=True)

    filename = dataset_name + '_' + str(alpha) + '_temp2SVM' + '.npy'
    with open(os.path.join(FILEPATH, filename), 'wb') as f:
        #np.save(f, np.concatenate([w_best, b_best]))
        np.save(f, torch.concat([w, b]).detach().numpy())

In [ ]:
C = 1e-2
num_epochs = 100

# losses_test = []

for alpha in ALPHA_GRID[:1]:

    w = torch.autograd.Variable(torch.randn(processed_data.shape[1]), requires_grad=True)
    b = torch.autograd.Variable(torch.rand(1), requires_grad=True)

    w = torch.autograd.Variable(torch.randn(processed_data.shape[1]), requires_grad=True)



    # optimizer = torch.optim.SGD([w, b], lr=2e-1)
    # # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.95)
    # optimizer = gpflow.optimizers.Scipy()

    device = 'cpu'
    for epoch in tqdm(range(num_epochs)[:2]):
        for i, (fields, target, gr) in enumerate(train_data_loader):
            
            fields, target, gr = fields.to(device), target.to(device), gr.to(device)
            y = torch.matmul(fields.float(), w) + b
            mask = (gr==1)

            n1 = y[mask].size()[0]
            n = y.size()[0]
            w1 = n1 / n
            w2 = 1 - w1

            loss1 = torch.mean(torch.maximum(torch.zeros(n1), 1 - target[mask]*y[mask]))
            loss2 = torch.mean(torch.maximum(torch.zeros(n-n1), 1 - target[~mask]*y[~mask]))
            max_loss = torch.maximum(loss1, loss2)

            loss = max_loss + 1/alpha * torch.log(w1*torch.exp(alpha*(loss1-max_loss)) + w2*torch.exp(alpha*(loss2-max_loss))) + C/2*torch.dot(w, w) 

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            def func_to_min(bws,w1,w2,n,y,mask,target,alpha,C):
              b = torch.from_numpy(np.array(bws[-1]))
              w = torch.from_numpy(bws[:-1])
              n1 = int(w1*n)
              loss1 = torch.mean(torch.maximum(torch.zeros(n1), 1 - target[mask]*y[mask]))
              loss2 = torch.mean(torch.maximum(torch.zeros(n-n1), 1 - target[~mask]*y[~mask]))
              # print(loss1, loss2)
              max_loss = torch.maximum(loss1, loss2)
              loss = max_loss + 1.0/alpha * torch.log(w1*torch.exp(alpha*(loss1-max_loss)) + w2*torch.exp(alpha*(loss2-max_loss))) + C/2*torch.dot(w, w) 
              return(loss)

            w.data -= w.grad.data # step
            b.data -= b.grad.data # step

            search_start = w.data[:].numpy()
            search_start = np.append(search_start, b.data[:].numpy())

            res = minimize(
                partial(func_to_min, w1=w1, w2=w2, n=n, y=y,mask=mask, alpha=alpha, C=C, target=target),
                x0=search_start
            )

            w.data = torch.from_numpy(res.x[:-1])
            b.data = torch.from_numpy(res.x[-1])

            # loss = max_loss + 1/alpha * torch.log(w1*torch.exp(alpha*(loss1-max_loss)) + w2*torch.exp(alpha*(loss2-max_loss))) + C/2*torch.dot(w, w) 
            #scheduler.step()

            #w.grad.data.zero_()
            #b.grad.data.zero_()
            # print(loss)
            # losses_test.append(loss)

    filename = dataset_name + '_' + str(alpha) + '_temp' + '.npy'
    with open(os.path.join(FILEPATH, filename), 'wb') as f:
        np.save(f, torch.concat([w, b]).detach().numpy())

In [ ]:

# def run_optimizer_on_minibatch_size(model, iterations, minibatch_size, xAndY):
#     """
#     Utility function running a Scipy optimizer
#     :param model: GPflow model
#     :param interations: number of iterations
#     """
#     N = xAndY[0].shape[0]
#     tensor_data = tuple(map(tf.convert_to_tensor, xAndY))
#     train_dataset = tf.data.Dataset.from_tensor_slices(tensor_data).repeat().shuffle(N)
    
#     logf = []
#     train_iter = iter(train_dataset.batch(minibatch_size))
#     training_loss = model.training_loss_closure(train_iter, compile=True)
#     optimizer = gpflow.optimizers.Scipy()

#     @tf.function   # had to remove this decorator
#     def optimization_step():
#         optimizer.minimize(training_loss, model.trainable_variables)

#     # step = 0
#     for step in range(iterations):
#         optimization_step()
#         if step % 10 == 0:
#             elbo = -training_loss().numpy()
#             logf.append(elbo)
#             print(elbo)
#     return logf

# from gpflow.ci_utils import ci_niter
# maxiter = ci_niter(20000)
# logf = run_optimizer_on_minibatch_size(m, maxiter, minibatch_size, (X,Y))

In [ ]:
def measure_performance(filenames, skip, isTrain=True):
    
    df = pd.DataFrame(columns=['alpha', 'loss', 'disp', 'accuracy'])

    for i, filename in enumerate(filenames):
        with open(os.path.join(FILEPATH, filename), 'rb') as f:
            params = np.load(f)
        w = params[:-1]
        b = params[-1]

        alpha = float(filename.split('/')[-1][skip:].split('_')[0])

        if isTrain:
            X = train_dataset.dataset.data[train_dataset.indices]
            target = train_dataset.dataset.targets[train_dataset.indices]
            gr = train_dataset.dataset.groups[train_dataset.indices]
        else:
            X = test_dataset.dataset.data[test_dataset.indices]
            target = test_dataset.dataset.targets[test_dataset.indices]
            gr = test_dataset.dataset.groups[test_dataset.indices]

        mask = (gr == 1)
        n1 = np.sum(mask)
        n2 = np.sum(~mask)

        y_ = np.dot(X, w) + b
        loss1 = np.mean(np.maximum(np.zeros(n1), 1 - target[mask]*y_[mask]))
        loss2 = np.mean(np.maximum(np.zeros(n2), 1 - target[~mask]*y_[~mask]))

        correct = np.sum(np.sign(y_).astype(int) == target)
        accuracy = correct / len(y_)

        w1 = n1/(n1+n2)
        w2 = 1 - w1

        df.loc[i] = [alpha, w1*loss1 + w2*loss2, np.abs(loss1 - loss2), accuracy]
    
        # print(w1*loss1 + w2*loss2)

    df.sort_values(by=['alpha'], inplace=True)
    return df

In [ ]:
file_list

In [ ]:
file_list = [x for x in os.listdir(FILEPATH) if ('temp' in x ) and ('loan' in x)]

In [ ]:
skip_symbols = len(dataset_name + '_')
res = measure_performance(file_list, skip=skip_symbols, isTrain=True)

In [ ]:
plt.scatter(res['loss'].values, res['disp'].values, s=20+5*np.arange(len(res)))

In [ ]:
svc = LinearSVC(max_iter=5000, C=1)

In [ ]:
svc.fit(train_dataset.dataset.data[train_dataset.indices], train_dataset.dataset.targets[train_dataset.indices])

In [ ]:
svc.score(train_dataset.dataset.data[train_dataset.indices], train_dataset.dataset.targets[train_dataset.indices])

In [ ]:
np.sum(svc.coef_**2)

In [ ]:
np.corrcoef(w.detach().numpy(), svc.coef_.flatten())

In [ ]:
torch.sum(w**2)

In [ ]:
"""
    Author: Lasse Regin Nielsen
"""

from __future__ import division, print_function
import os
import numpy as np
import random as rnd
# filepath = os.path.dirname(os.path.abspath(__file__))
filepath ="/content/drive/MyDrive/svm-smo/"


class SVM():
    """
        Simple implementation of a Support Vector Machine using the
        Sequential Minimal Optimization (SMO) algorithm for training.
    """
    def __init__(self, max_iter=10000, kernel_type='linear', C=1.0, epsilon=0.001):
        self.kernels = {
            'linear' : self.kernel_linear,
            'quadratic' : self.kernel_quadratic
        }
        self.max_iter = max_iter
        self.kernel_type = kernel_type
        self.C = C
        self.epsilon = epsilon
        
    def fit(self, X, y):
        # Initialization
        n, d = X.shape[0], X.shape[1]
        alpha = np.zeros((n))
        kernel = self.kernels[self.kernel_type]
        count = 0
        while True:
            count += 1
            alpha_prev = np.copy(alpha)
            for j in range(0, n):
                i = self.get_rnd_int(0, n-1, j) # Get random int i~=j
                x_i, x_j, y_i, y_j = X[i,:], X[j,:], y[i], y[j]
                k_ij = kernel(x_i, x_i) + kernel(x_j, x_j) - 2 * kernel(x_i, x_j)
                if k_ij == 0:
                    continue
                alpha_prime_j, alpha_prime_i = alpha[j], alpha[i]
                (L, H) = self.compute_L_H(self.C, alpha_prime_j, alpha_prime_i, y_j, y_i)

                # Compute model parameters
                self.w = self.calc_w(alpha, y, X)
                self.b = self.calc_b(X, y, self.w)

                # Compute E_i, E_j
                E_i = self.E(x_i, y_i, self.w, self.b)
                E_j = self.E(x_j, y_j, self.w, self.b)

                # Set new alpha values
                alpha[j] = alpha_prime_j + float(y_j * (E_i - E_j))/k_ij
                alpha[j] = max(alpha[j], L)
                alpha[j] = min(alpha[j], H)

                alpha[i] = alpha_prime_i + y_i*y_j * (alpha_prime_j - alpha[j])

            # Check convergence
            diff = np.linalg.norm(alpha - alpha_prev)
            if diff < self.epsilon:
                break

            if count >= self.max_iter:
                print("Iteration number exceeded the max of %d iterations" % (self.max_iter))
                return
        # Compute final model parameters
        self.b = self.calc_b(X, y, self.w)
        if self.kernel_type == 'linear':
            self.w = self.calc_w(alpha, y, X)
        # Get support vectors
        alpha_idx = np.where(alpha > 0)[0]
        support_vectors = X[alpha_idx, :]
        return support_vectors, count
    def predict(self, X):
        return self.h(X, self.w, self.b)
    def calc_b(self, X, y, w):
        b_tmp = y - np.dot(w.T, X.T)
        return np.mean(b_tmp)
    def calc_w(self, alpha, y, X):
        return np.dot(X.T, np.multiply(alpha,y))
    # Prediction
    def h(self, X, w, b):
        return np.sign(np.dot(w.T, X.T) + b).astype(int)
    # Prediction error
    def E(self, x_k, y_k, w, b):
        return self.h(x_k, w, b) - y_k
    def compute_L_H(self, C, alpha_prime_j, alpha_prime_i, y_j, y_i):
        if(y_i != y_j):
            return (max(0, alpha_prime_j - alpha_prime_i), min(C, C - alpha_prime_i + alpha_prime_j))
        else:
            return (max(0, alpha_prime_i + alpha_prime_j - C), min(C, alpha_prime_i + alpha_prime_j))
    def get_rnd_int(self, a,b,z):
        i = z
        cnt=0
        while i == z and cnt<1000:
            i = rnd.randint(a,b)
            cnt=cnt+1
        return i
    # Define kernels
    def kernel_linear(self, x1, x2):
        return np.dot(x1, x2.T)
    def kernel_quadratic(self, x1, x2):
        return (np.dot(x1, x2.T) ** 2)


In [ ]:

from __future__ import division, print_function
import csv, os, sys
import numpy as np
import pandas as pd
# from SVM import SVM
# filepath ="/content/drive/MyDrive/svm-smo/"
os.chdir(filepath)


#  filepath = os.path.dirname(os.path.abspath(__file__))


def readData(filename, header=True):

    csvfile = pd.read_csv(filename, encoding='utf-8')
    # data, header = [], None
    # with open(filename, 'rb') as csvfile:
    #     spamreader = csv.reader(csvfile, delimiter=',')
    #     if header:
    #         header = spamreader.next()
    #     for row in spamreader:
    #         data.append(row)
    return (np.array(csvfile.to_numpy()), np.asarray(csvfile.columns.values))

def calc_acc(y, y_hat):
    idx = np.where(y_hat == 1)
    TP = np.sum(y_hat[idx] == y[idx])
    idx = np.where(y_hat == -1)
    TN = np.sum(y_hat[idx] == y[idx])
    return float(TP + TN)/len(y)

def main(filename='data/iris-virginica.txt', C=1.0, kernel_type='linear', epsilon=0.001):
    # Load data
    (data, _) = readData('%s/%s' % (filepath, filename), header=False)
    data = data.astype(float)

    # Split data
    X, y = data[:,0:-1], data[:,-1].astype(int)

    # Initialize model
    model = SVM()

    # Fit model
    support_vectors, iterations = model.fit(X, y)

    # Support vector count
    sv_count = support_vectors.shape[0]

    # Make prediction
    y_hat = model.predict(X)

    # Calculate accuracy
    acc = calc_acc(y, y_hat)

    print("Support vector count: %d" % (sv_count))
    print("bias:\t\t%.3f" % (model.b))
    print("w:\t\t" + str(model.w))
    print("accuracy:\t%.3f" % (acc))
    print("Converged after %d iterations" % (iterations))

if __name__ == '__main__':
    if ('--help' in sys.argv) or ('-h' in sys.argv):
        print("")
        print("Trains a support vector machine.")
        print("Usage: %s FILENAME C kernel eps" % (sys.argv[0]))
        print("")
        print("FILENAME: Relative path of data file.")
        print("C:        Value of regularization parameter C.")
        print("kernel:   Kernel type to use in training.")
        print("          'linear' use linear kernel function.")
        print("          'quadratic' use quadratic kernel function.")
        print("eps:      Convergence value.")
    else:
        kwargs = {}
        # if len(sys.argv) > 1:
        #     kwargs['filename'] = sys.argv[1]
        # if len(sys.argv) > 2:
        #     kwargs['C'] = float(sys.argv[2])
        # if len(sys.argv) > 3:
        #     kwargs['kernel_type'] = sys.argv[3]
        # if len(sys.argv) > 4:
        #     kwargs['epsilon'] = float(sys.argv[4])
        # if len(sys.argv) > 5:
        #     sys.exit("Not correct arguments provided. Use %s -h for more information"
        #              % (sys.argv[0]))
        main(**kwargs)


In [ ]:
'%s/%s' % (filepath, filename)

In [ ]:
# svc.fit(train_dataset.dataset.data[train_dataset.indices], train_dataset.dataset.targets[train_dataset.indices])

In [ ]:
train_dataset.dataset.data[train_dataset.indices].shape

In [ ]:
train_dataset.dataset.targets[train_dataset.indices].astype(int)

In [ ]:
iris = pd.read_csv('%s/%s' % (filepath, 'data/iris-virginica.txt'), encoding='utf-8')

In [ ]:
len(train_dataset.indices)

In [ ]:
X, y = (train_dataset.dataset.data[train_dataset.indices[:500]], train_dataset.dataset.targets[train_dataset.indices[:500]].astype(int))
print(X.shape,y.shape)

In [ ]:
X, y = (train_dataset.dataset.data[train_dataset.indices[2000:2500]], train_dataset.dataset.targets[train_dataset.indices[2000:2500]].astype(int))

model = SVM(max_iter=500)

# Fit model
support_vectors, iterations = model.fit(X, y)
# Support vector count
sv_count = support_vectors.shape[0]
# Make prediction
y_hat = model.predict(X)
# Calculate accuracy
acc = calc_acc(y, y_hat)

print("Support vector count: %d" % (sv_count))
print("bias:\t\t%.3f" % (model.b))
print("w:\t\t" + str(model.w))
print("accuracy:\t%.3f" % (acc))
print("Converged after %d iterations" % (iterations))


In [ ]:
svc = LinearSVC(max_iter=5000, C=1)

In [ ]:
svc.fit(train_dataset.dataset.data[train_dataset.indices[:1500]], train_dataset.dataset.targets[train_dataset.indices[:1500]])

In [ ]:
svc.score(train_dataset.dataset.data[train_dataset.indices[:1500]], train_dataset.dataset.targets[train_dataset.indices[:1500]])

In [ ]:
C = 1e-2
num_epochs = 100

for alpha in ALPHA_GRID:

    w = torch.autograd.Variable(torch.randn(processed_data.shape[1]), requires_grad=True)
    b = torch.autograd.Variable(torch.rand(1), requires_grad=True)
    optimizer = scipy.optim.SGD([w, b], lr=2e-1)

    
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.95)

    device = 'cpu'
    for epoch in tqdm(range(num_epochs)[:2]):
        for i, (fields, target, gr) in enumerate(train_data_loader):


            fields, target, gr = fields.to(device), target.to(device), gr.to(device)

            y = torch.matmul(fields.float(), w) + b
            mask = (gr==1)

            n1 = y[mask].size()[0]
            n = y.size()[0]
            w1 = n1 / n
            w2 = 1 - w1

            loss1 = torch.mean(torch.maximum(torch.zeros(n1), 1 - target[mask]*y[mask]))
            loss2 = torch.mean(torch.maximum(torch.zeros(n-n1), 1 - target[~mask]*y[~mask]))
            max_loss = torch.maximum(loss1, loss2)

            loss = max_loss + 1/alpha * torch.log(w1*torch.exp(alpha*(loss1-max_loss)) + w2*torch.exp(alpha*(loss2-max_loss))) + C/2*torch.dot(w, w) 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #scheduler.step()
            #w.data -= step_size * w.grad.data # step
            #b.data -= step_size * b.grad.data # step

            #w.grad.data.zero_()
            #b.grad.data.zero_()
    filename = dataset_name + '_' + str(alpha) + '_temp' + '.npy'
    with open(os.path.join(FILEPATH, filename), 'wb') as f:
        np.save(f, torch.concat([w, b]).detach().numpy())

In [ ]:
C = 1e-2
num_epochs = 100

for alpha in ALPHA_GRID:

    w = torch.autograd.Variable(torch.randn(processed_data.shape[1]), requires_grad=True)
    b = torch.autograd.Variable(torch.rand(1), requires_grad=True)
    optimizer = torch.optim.SGD([w, b], lr=2e-1)

    
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.95)

    device = 'cpu'
    for epoch in tqdm(range(num_epochs)[:2]):
        for i, (fields, target, gr) in enumerate(train_data_loader):


            fields, target, gr = fields.to(device), target.to(device), gr.to(device)

            y = torch.matmul(fields.float(), w) + b
            mask = (gr==1)

            n1 = y[mask].size()[0]
            n = y.size()[0]
            w1 = n1 / n
            w2 = 1 - w1

            loss1 = torch.mean(torch.maximum(torch.zeros(n1), 1 - target[mask]*y[mask]))
            loss2 = torch.mean(torch.maximum(torch.zeros(n-n1), 1 - target[~mask]*y[~mask]))
            max_loss = torch.maximum(loss1, loss2)

            loss = max_loss + 1/alpha * torch.log(w1*torch.exp(alpha*(loss1-max_loss)) + w2*torch.exp(alpha*(loss2-max_loss))) + C/2*torch.dot(w, w) 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #scheduler.step()
            #w.data -= step_size * w.grad.data # step
            #b.data -= step_size * b.grad.data # step

            #w.grad.data.zero_()
            #b.grad.data.zero_()
    filename = dataset_name + '_' + str(alpha) + '_temp' + '.npy'
    with open(os.path.join(FILEPATH, filename), 'wb') as f:
        np.save(f, torch.concat([w, b]).detach().numpy())